In [1]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime
import json 

In [19]:
file = "./NY/ny_simplified.geojson"
state_boundary = gp.read_file(file)

In [ ]:
reader = csv.reader(open('./NY_county.csv', 'r'))
county_fips = {}
for row in reader:
    key = row[5]
    value = row[0]
    county_fips[key] = value

In [20]:
state_boundary.head()

,id,canonical,type,centerX,centerY,neighbors,geometry
0,36001263,NY Albany 263,ORIGINAL,+42.7272097,-073.7873353,"36001262,36001253,36001247,36001252,36001265,3...","MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,NY Albany 232,ORIGINAL,+42.7295842,-073.7227157,"36001243,36001249,36001343,36001344,36001251,3...","MULTIPOLYGON (((-73.71364 42.72799, -73.70998 ..."
2,36001342,NY Albany 342,ORIGINAL,+42.7348777,-073.7093285,"36001233,36001243,36001340,36001341,36001339,3...","MULTIPOLYGON (((-73.71210 42.73705, -73.71182 ..."
3,36001341,NY Albany 341,ORIGINAL,+42.7355901,-073.7037615,"36001233,36001342,36001339,36001285","MULTIPOLYGON (((-73.70277 42.73421, -73.70857 ..."
4,36001277,NY Albany 277,ORIGINAL,+42.7313271,-073.8385576,"36001275,36001276,36001273","MULTIPOLYGON (((-73.82521 42.72768, -73.82606 ..."


In [ ]:
#Rename columns
state_boundary.rename(columns = {"GEOID10":"id"}, inplace = True)
state_boundary.rename(columns = {"INTPTLAT10":"centerX"}, inplace = True)
state_boundary.rename(columns = {"INTPTLON10":"centerY"}, inplace = True)
state_boundary.rename(columns = {"NEIGHBORS":"neighbors"}, inplace = True)

#Add type columns
state_boundary.insert(1, 'canonical', None)
state_boundary.insert(2, 'type', 'ORIGINAL') 

#Drop column
#df = df.drop (columns = ['ALAND10'])
state_boundary.head()


In [ ]:
#Canonical name
STATE = 'NY'
for i, row in ny_boundary.iterrows():   
    id = state_boundary.at[i, 'id']
    county_id = id[ : 5]
    canonical_id = state + ' ' + county_fips[county_id] + ' ' + id[5:]
    state_boundary.at[i, 'canonical'] = canonical_id
    
    #print (canonicalId)
state_boundary.head()

In [ ]:
#Replace multipoly with polygon
arr_polygon = [0] * len(df)
arr_polygon = gp.GeoDataFrame (arr_polygon, columns = ['geometry'])
for index, row in df.iterrows():
    #Progress
    if index % 1000 == 0:
        print (index)
    polygon = row.geometry[0]
    arr_polygon.loc[index] = polygon
    
df['geometry'] = arr_polygon
df.head()    

In [21]:
state_boundary.to_file("./NY/NY.geojson", driver='GeoJSON')

In [ ]:
### GENERATE NEIGHBORS

file = "../data/new/NY_precinct_boundary.geojson"  
state_boundary = gp.read_file(file) # open file

state_boundary["neighbors"] = None  # add NEIGHBORS column

for index, row in state_boundary.iterrows():
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
        
    neighbors = state_boundary[~state_boundary.geometry.disjoint(row.geometry)].id.tolist()
    # remove own name from the list
    neighbors = [ name for name in neighbors if row.id != name ]
    # add names of neighbors as NEIGHBORS value
    state_boundary.at[index, 'neighbors'] = ", ".join(neighbors)

state_boundary.to_file(".NY/NY_neighbors.geojson", driver='GeoJSON')

In [16]:
def create_neighbors_df(neighbors, df):
    neighbors = neighbors.split(',')
    neighbors_df = df[df['id'].isin(neighbors)][['id', 'geometry']]
    return neighbors_df 

In [14]:
#INTERSECTIONS

file = "./NY/NY.geojson"  
precincts = gp.read_file(file) # open file
precincts = precincts.drop(columns=['type', 'centerX', 'centerY', 'canonical'])
precincts.head()

,id,neighbors,geometry
0,36001263,"36001262,36001253,36001247,36001252,36001265,3...","MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,"36001243,36001249,36001343,36001344,36001251,3...","MULTIPOLYGON (((-73.71364 42.72799, -73.71119 ..."
2,36001342,"36001233,36001243,36001340,36001341,36001339,3...","MULTIPOLYGON (((-73.71210 42.73705, -73.71182 ..."
3,36001341,"36001233,36001342,36001339,36001285","MULTIPOLYGON (((-73.70277 42.73421, -73.70857 ..."
4,36001277,"36001275,36001276,36001273","MULTIPOLYGON (((-73.82521 42.72768, -73.82606 ..."


In [17]:
AREA_THRESHOLD = 1e-4
intersections = gp.GeoDataFrame(crs = precincts.crs)

for index, row in precincts.iterrows():
    #For progress
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
    neighbors = create_neighbors_df(row['neighbors'], precincts)
    #print(neighbors)
    # check if intersection occured
    overlaps = neighbors[neighbors.geometry.overlaps(row.geometry)]['id'].tolist()
    if len(overlaps) > 0:
        # compare the area with threshold
        for y in overlaps:
            temp_area = gp.overlay(precincts.loc[precincts.id == y,], precincts.loc[precincts.id == row.id], how='intersection')
            temp_area = temp_area.loc[temp_area.geometry.area >= AREA_THRESHOLD]
            
            if temp_area.shape[0] > 0:
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections], ignore_index=True),crs = precincts.crs)
                intersections['sorted'] = intersections.apply(lambda y: sorted([y['id_1'],y['id_2']]), axis=1)
                intersections['sorted'] = intersections.sorted.apply(lambda y: ''.join(y))
                intersections = intersections.drop_duplicates('sorted')
                intersections = intersections.reset_index()[['id_1','id_2','geometry']]
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections],ignore_index=True),crs=precincts.crs)

17:03:54 0
17:04:40 1000
17:05:41 2000
17:06:38 3000
17:07:26 4000
17:08:05 5000
17:08:59 6000
17:10:00 7000
17:10:40 8000
17:11:31 9000
17:12:35 10000
17:13:19 11000
17:14:14 12000
17:15:14 13000
17:16:18 14000


In [18]:
intersections.shape

(0, 0)

In [ ]:
### GAPS

file = "./NY/NY_precinct_boundary_neighbors_full.geojson"
precincts = gp.read_file(file)

precincts_diss = precincts[['geometry', 'type']]
precincts_diss = precincts_diss.dissolve(by='type')
#precincts_diss = df.drop (columns = ['type', 'id', 'canonical', 'centerX', 'centerY', 'area'])

precincts_diss.head()

In [ ]:
interior = precincts_diss.interiors.values.tolist()

gaps = []
for i in interior:
    if i == []:
        continue
    #print(i[0].coords[:])    
    gaps.append(Polygon(i[0]))
    
gaps = gp.GeoDataFrame(geometry = gaps, crs = precincts.crs)
gaps['feature_touches'] = gaps.geometry.apply(lambda y: precincts.loc[precincts.touches(y)]['id'].tolist())
gaps.head()

In [ ]:
gaps.to_file("../data/new/NY_overlap.geojson", driver='GeoJSON')

In [27]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [28]:
file = "./NY/NY.geojson"
precincts = gp.read_file(file)
precincts.head()


,id,canonical,type,centerX,centerY,neighbors,geometry
0,36001263,NY Albany 263,ORIGINAL,+42.7272097,-073.7873353,"36001262,36001253,36001247,36001252,36001265,3...","MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,NY Albany 232,ORIGINAL,+42.7295842,-073.7227157,"36001243,36001249,36001343,36001344,36001251,3...","MULTIPOLYGON (((-73.71364 42.72799, -73.70998 ..."
2,36001342,NY Albany 342,ORIGINAL,+42.7348777,-073.7093285,"36001233,36001243,36001340,36001341,36001339,3...","MULTIPOLYGON (((-73.71210 42.73705, -73.71182 ..."
3,36001341,NY Albany 341,ORIGINAL,+42.7355901,-073.7037615,"36001233,36001342,36001339,36001285","MULTIPOLYGON (((-73.70277 42.73421, -73.70857 ..."
4,36001277,NY Albany 277,ORIGINAL,+42.7313271,-073.8385576,"36001275,36001276,36001273","MULTIPOLYGON (((-73.82521 42.72768, -73.82606 ..."


In [29]:
add_precinct_querry = ("INSERT INTO mhaczynska.precinct "
               "(GEO_ID, NAME, STATE_NAME, CENTER_X, CENTER_Y, NEIGHBORS, PRECINCT_GEOJSON) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s)")

#update_precinct_querry = "UPDATE mhaczynska.precinct SET CENTER_X = %s, CENTER_Y = %s WHERE GEO_ID = %s"

state = 'NY'
#precinct_data = []
for index, row in precincts.iterrows():
    geometry_str = row.geometry.__geo_interface__
    geometry_str = json.dumps(geometry_str)     
    precinct_data = (row['id'], row['canonical'], state, row['centerX'], row['centerY'], row['neighbors'], geometry_str)
    #precinct_data.append(precinct_row)
    #update_precinct = (row['centerX'], row['centerY'], row['id'])
    cursor.execute(add_precinct_querry, precinct_data)
    mydb.commit()
    

In [10]:
#ADD NEIGHBORS
add_neighbor_querry = ("INSERT INTO mhaczynska.neighbors "
               "(PRECINCT, NEIGHBOR_GEOID) "
               "VALUES (%s, %s)")

neighbor_data = []
for index, row in precincts.iterrows():
    neighbors = row['neighbors']
    neighbors = neighbors.split(",")    
    for neighbor in neighbors:
        neighbor_row = (row['id'], neighbor.strip())
        neighbor_data.append(neighbor_row)
cursor.executemany(add_neighbor_querry, neighbor_data)
mydb.commit()

In [30]:
cursor.close()
mydb.close()